In [1]:
import pandas as pd
from numpy import genfromtxt
from PIL import Image
import torch
import numpy as np
from copy import deepcopy as copy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [2]:
class Net(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 28, 3)
            self.pool1 = nn.MaxPool2d((2, 2))
            self.fc1 = nn.Linear(4732, 10)
        
        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = self.pool1(x)
            x = x.flatten(start_dim=1)
            x = self.fc1(x)
            return F.softmax(x, dim=1)

In [4]:
device = torch.device('cuda:0')
net = Net().to(device)
net.load_state_dict(torch.load('model1'))
net.eval()

Net(
  (conv1): Conv2d(1, 28, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4732, out_features=10, bias=True)
)

In [7]:
test = genfromtxt('../data/digit-recognizer/test.csv', delimiter=',')[1:]

In [8]:
np.save('testing_data.npy', test)

In [6]:
test = np.load('testing_data.npy', allow_pickle=True)

In [10]:
BATCH_SIZE = 100
output = []
for i in range(0, len(test), BATCH_SIZE):
    batch_X = torch.Tensor(test[i:i+BATCH_SIZE]).view(-1, 1, 28, 28).to(device)
    output.append(net(batch_X))

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 4.40 GiB already allocated; 0 bytes free; 4.41 GiB reserved in total by PyTorch)

In [5]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0) 
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t)
print(r)
print(a)
print(f)

6442450944
2097152
191488
1905664
